In [1]:
import pandas as pd
from gpt_sender_async import GPT_API as gpt
import asyncio
from tqdm import tqdm
import json

In [3]:
df = pd.read_csv("new_data_v2.csv", index_col=0)
# df = df[df["Предложенная Точка отказа"].isna()]

In [6]:
df["Точка отказа"].value_counts()

Блок питания               380
Матрица                    360
Материнская плата          300
Диск                       140
Консультация               120
Вентилятор                  60
Клавиатура                  60
Оперативная память          40
Wi-fi модуль                40
Wi-fi антенна               40
Программное обеспечение     30
Сервер                      20
Корпус                      10
Динамики                    10
Камера                      10
Аккумулятор                 10
SFP модуль                  10
Jack                        10
Name: Точка отказа, dtype: int64

In [7]:
small_amount =  ["Оперативная память", "Wi-fi модуль", "Wi-fi антенна", 
                 "Программное обеспечение", "Сервер", "Корпус", "Динамики", "Камера",
                 "Аккумулятор", "SFP модуль", "Jack"]

In [8]:
df.head(2)

,Тема,Описание,Тип оборудования,Точка отказа,Серийный номер
0,Камера не работает,"Заметил, что камера на ноутбуке перестала вклю...",Ноутбук,Камера,NaN
1,Проблемы с камерой,"Добрый день! Недавно столкнулся с проблемой, ч...",Ноутбук,Камера,NaN


In [44]:
def get_prompt(tema, opisanie, oborudovanie, tochka_otkaza):
    return '''"""
Тема: {0}
Описание: {1}
Оборудование: {2}
Точка отказа: {3}
"""
Выше представлено обращение пользователя в тех поддержку с темой, описанием и проблемой.
Сгенерируй 5 разных похожих обращений, опираясь на точку отказа и оборудование на котором произошел отказ.
Обращения должны сильно отличаться друг от друга, представь что ты пользователь и пиши обращения исходя из этой роли.
Упор делай что проблема в "{3}", а не в чем что-еще.

Верни в формате json. 
Пример: {{1 : {{"Тема": str, "Описание": str}}}}
Где вместо str должны быть тема и описание проблемы и так для 5 обращений. Return only JSON.
'''.format(tema, opisanie, oborudovanie, tochka_otkaza)
    

In [45]:
async def get_responses(prompts):
    tasks = [gpt.get_response(prompt, json_output=True) 
             for prompt in prompts]
    responses = await asyncio.gather(*tasks)
    return responses


In [46]:
batch_size = 8
result = []
for i in tqdm(range(0, len(df), batch_size)):
    # Add the batch to the batches list.

    data = []
    for index, row in df[i:i+batch_size].iterrows():

        data.append([row["Тема"], row["Описание"], row["Тип оборудования"], row["Точка отказа"]])

    summaries = await get_responses([get_prompt(*row) for row in data])

    for row, summary in zip(data, summaries):

        if summary is None:
            continue
        
        result.append({"Тип оборудования": row[2], "Точка отказа": row[3], "Обращения": summary})

100%|██████████| 207/207 [24:39<00:00,  7.15s/it]


In [48]:
result

[{'Тип оборудования': 'Ноутбук',
  'Точка отказа': 'Камера',
  'Обращения': {'1': {'Тема': 'Проблема с веб-камерой',
    'Описание': 'Не могу запустить камеру на ноутбуке во время онлайн-уроков. Она просто не реагирует, пробовал перезагружать устройство и обновлять драйвера.'},
   '2': {'Тема': 'Камера ноутбука не функционирует',
    'Описание': 'На моем ноутбуке веб-камера не работает. Пробовал разные приложения для видеозвонков, но камера не включается.'},
   '3': {'Тема': 'Неполадки с камерой',
    'Описание': 'У моего ноутбука возникла проблема: камера не работает на видеоконференциях. Испробовал множество способов, включая установку новых драйверов, но это не помогло.'},
   '4': {'Тема': 'Не работает встроенная камера',
    'Описание': 'Встроенная веб-камера на ноутбуке перестала включаться. Пытался использовать ее в Skype и Zoom, но получается только черный экран.'},
   '5': {'Тема': 'Вопрос по камере ноутбука',
    'Описание': 'Наткнулся на проблему с камерой на своем ноутбуке. 

In [49]:
with open("new_data_gpt4o_v2.jsonl", 'w') as file:
    for entry in result:
        json_line = json.dumps(entry,ensure_ascii=False)  # Convert dictionary to JSON string
        file.write(json_line + '\n')   